<a href="https://colab.research.google.com/github/69-LaxmiArsul/Smartknower-MajorProject/blob/main/Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
df=pd.read_csv('/tripadvisor_hotel_reviews.csv')
df

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",5
20487,great location price view hotel great quick pl...,4
20488,"ok just looks nice modern outside, desk staff ...",2
20489,hotel theft ruined vacation hotel opened sept ...,1


In [9]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

vect=CountVectorizer()
docs=np.array(['hi my name is xyz'
              'my favourite color is blue'
              'blue is the color of sky']) 

bag=vect.fit_transform(docs)              

In [11]:
print(vect.vocabulary_)

{'hi': 3, 'my': 5, 'name': 6, 'is': 4, 'xyzmy': 10, 'favourite': 2, 'color': 1, 'blueblue': 0, 'the': 9, 'of': 7, 'sky': 8}


In [12]:
print(bag.toarray())

[[1 2 1 1 3 1 1 1 1 1 1]]


In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
np.set_printoptions(precision=2)
tfidf=TfidfTransformer(use_idf=True,norm='l2',smooth_idf=True)
print(tfidf.fit_transform(bag).toarray())

[[0.21 0.43 0.21 0.21 0.64 0.21 0.21 0.21 0.21 0.21 0.21]]


In [15]:
import nltk
nltk.download('stopwords')
stopword_list=nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
!pip install contractions

     |████████████████████████████████| 286kB 30.8MB/s 
     |████████████████████████████████| 327kB 43.0MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85395 sha256=8d8bec4f631a17bb85c518a37ab31a60bd5057bd8d1a5925c54ca19492d6a303
  Stored in directory: /root/.cache/pip/wheels/3a/03/34/77e3ece0bba8b86bfac88a79f923b36d805cad63caeba38842
Successfully built pyahocorasick


In [49]:
import contractions
def con(text):
  expand=contractions.fix(text)
  return expand  

In [50]:
import re
def remove_sp(text):
  pattern=r'[^A-Za-z\s]'
  text=re.sub(pattern,'',text)
  return text

In [51]:
from nltk.tokenize.toktok import ToktokTokenizer
tokenizer=ToktokTokenizer()

In [52]:
def remove_stopwords(text):
  tokens=tokenizer.tokenize(text)
  tokens=[token.strip() for token in tokens]
  filtered_tokens=[token for token in tokens if token not in stopword_list]
  filtered_text=' '.join(filtered_tokens)
  return filtered_text 

In [54]:
df.Review=df.Review.apply(lambda x:x.lower())
df.Review=df.Review.apply(con)
df.Review=df.Review.apply(remove_sp)
df.Review=df.Review.apply(remove_stopwords)

In [55]:
df.head

<bound method NDFrame.head of                                                   Review  Rating
0      nice hotel expensive parking got good deal sta...       4
1      ok nothing special charge diamond member hilto...       2
2      nice rooms experience hotel monaco seattle goo...       3
3      unique great stay wonderful time hotel monaco ...       5
4      great stay great stay went seahawk game awesom...       5
...                                                  ...     ...
20486  best kept secret rd time staying charm star ca...       5
20487  great location price view hotel great quick pl...       4
20488  ok looks nice modern outside desk staff nt par...       2
20489  hotel theft ruined vacation hotel opened sept ...       1
20490  people talking ca nt believe excellent ratings...       2

[20491 rows x 2 columns]>

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(
                      use_idf=True,
                      norm='l2',
                      smooth_idf=True
                      )

y = df.Rating.values
x=tfidf.fit_transform(df['Review'].values.astype('U'))


In [58]:
from sklearn.model_selection import train_test_split

In [59]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=1,test_size=0.5,shuffle=False)

In [62]:
from sklearn import svm
svm=svm.SVC(kernel='linear',probability=True)
prob=svm.fit(x_train,y_train).predict_proba(x_test)
y_pred_svm=svm.predict(x_test)

In [64]:
from sklearn.metrics import accuracy_score
print("Accuracy:",accuracy_score(y_test,y_pred_svm)*100,'%')

Accuracy: 61.50692953347647 %


In [60]:
import pickle
from sklearn.linear_model import LogisticRegressionCV
clf=LogisticRegressionCV(cv=5,
                         scoring='accuracy',
                         random_state=0,
                         n_jobs=-1,
                         verbose=3,
                         max_iter=300).fit(x_train,y_train)

saved_model=open('saved_model.sav','wb')
pickle.dump(clf,saved_model)
saved_model.close()                         

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:  9.1min finished


In [61]:
filename='saved_model.sav'
saved_clf=pickle.load(open(filename,'rb'))

saved_clf.score(x_test,y_test)

0.6099941440562171